# example with xgboost

Test XGBoost after it was compiled, pickle, unpickle.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

This is an example taken from [xgboost website](https://github.com/dmlc/xgboost).

In [3]:
import pickle

import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import load_iris, load_digits, load_boston

In [4]:
import xgboost as xgb

### Zeros and Ones from the Digits dataset: binary classification

In [5]:
rng = np.random.RandomState(31337)

digits = load_digits(2)
y = digits['target']
X = digits['data']
conf = []
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X, y):
    xgb_model = xgb.XGBClassifier().fit(X[train_index],y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    conf.append(confusion_matrix(actuals, predictions))
conf

[array([[87,  0],
        [ 1, 92]], dtype=int64), array([[91,  0],
        [ 3, 86]], dtype=int64)]

### Iris: multiclass classification

In [7]:
iris = load_iris()
y = iris['target']
X = iris['data']
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
conf = []
for train_index, test_index in kf.split(X, y):
    xgb_model = xgb.XGBClassifier().fit(X[train_index],y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    conf.append(confusion_matrix(actuals, predictions))
conf

[array([[19,  0,  0],
        [ 0, 31,  3],
        [ 0,  1, 21]], dtype=int64), array([[31,  0,  0],
        [ 0, 16,  0],
        [ 0,  3, 25]], dtype=int64)]

### Boston Housing: regression

In [9]:
boston = load_boston()
y = boston['target']
X = boston['data']
err = []
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X, y):
    xgb_model = xgb.XGBRegressor().fit(X[train_index],y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    err.append(mean_squared_error(actuals, predictions))
err

[9.862814662804583, 15.989963927018605]

### Parameter optimization

In [11]:
import joblib  # to check you can parallelize GridSearchCV
y = boston['target']
X = boston['data']
xgb_model = xgb.XGBRegressor()
clf = GridSearchCV(xgb_model,
                   {'max_depth': [2,4,6],
                    'n_estimators': [50,100,200]}, verbose=1, n_jobs=1, pre_dispatch=1)
clf.fit(X,y)
clf.best_score_, clf.best_params_

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    2.0s finished


(0.5984879207166734, {'max_depth': 4, 'n_estimators': 100})

### Pickling sklearn API models

In [13]:
# The sklearn API models are picklable
# must open in binary format to pickle
pickle.dump(clf, open("best_boston.pkl", "wb"))
clf2 = pickle.load(open("best_boston.pkl", "rb"))
np.allclose(clf.predict(X), clf2.predict(X))

True

### Early stopping

In [15]:
X = digits['data']
y = digits['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="auc",
        eval_set=[(X_test, y_test)])

[0]	validation_0-auc:0.999497
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.999497
[2]	validation_0-auc:0.999497
[3]	validation_0-auc:0.999749
[4]	validation_0-auc:0.999749
[5]	validation_0-auc:0.999749
[6]	validation_0-auc:0.999749
[7]	validation_0-auc:0.999749
[8]	validation_0-auc:0.999749
[9]	validation_0-auc:0.999749
[10]	validation_0-auc:1
[11]	validation_0-auc:1
[12]	validation_0-auc:1
[13]	validation_0-auc:1
[14]	validation_0-auc:1
[15]	validation_0-auc:1
[16]	validation_0-auc:1
[17]	validation_0-auc:1
[18]	validation_0-auc:1
[19]	validation_0-auc:1
[20]	validation_0-auc:1
Stopping. Best iteration:
[10]	validation_0-auc:1



XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)